In [96]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn import datasets, linear_model, metrics 
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import numpy as np
from math import sqrt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.pipeline import Pipeline
import xgboost as xgb

In [97]:
df = pd.read_csv("C:/Users/USER/Downloads/big_train.csv")
test = pd.read_csv("C:/Users/USER/Downloads/big_test.csv")

df_X = df.drop(['reg','Outlet_Establishment_Year'],axis=1)
#df_Y = df['Item_Outlet_Sales']

test_X = test.drop(['reg','Outlet_Establishment_Year'],axis=1)
#test_Y = test['Item_Outlet_Sales']

In [98]:
df_X['Unnamed: 0']
test_X['Unnamed: 0']

0          0
1          1
2          2
3          3
4          4
        ... 
5676    5676
5677    5677
5678    5678
5679    5679
5680    5680
Name: Unnamed: 0, Length: 5681, dtype: int64

In [99]:
number = LabelEncoder()
df_X['Item_Type'] = number.fit_transform(df_X['Item_Type'].astype(str))
#df_X['Outlet_Establishment_Year'] = number.fit_transform(df_X['Outlet_Establishment_Year'].astype(str))
df_X['Outlet_Size'] = number.fit_transform(df_X['Outlet_Size'].astype(str))
df_X['Outlet_Location_Type'] = number.fit_transform(df_X['Outlet_Location_Type'].astype(str))
df_X['Outlet_Type'] = number.fit_transform(df_X['Outlet_Type'].astype(str))
df_X['Outlet_Identifier'] = number.fit_transform(df_X['Outlet_Identifier'].astype(str))
#df_X['Item_Identifier'] = number.fit_transform(df_X['Item_Identifier'].astype(str))
#df_X['Item_Identifier'] = number.fit_transform(df_X['Item_Identifier'].astype(str))

number = LabelEncoder()
test_X['Item_Type'] = number.fit_transform(test_X['Item_Type'].astype(str))
#test_X['Outlet_Establishment_Year'] = number.fit_transform(test_X['Outlet_Establishment_Year'].astype(str))
test_X['Outlet_Size'] = number.fit_transform(test_X['Outlet_Size'].astype(str))
test_X['Outlet_Location_Type'] = number.fit_transform(test_X['Outlet_Location_Type'].astype(str))
test_X['Outlet_Type'] = number.fit_transform(test_X['Outlet_Type'].astype(str))
test_X['Outlet_Identifier'] = number.fit_transform(test_X['Outlet_Identifier'].astype(str))
#test_X['Item_Identifier'] = number.fit_transform(test_X['Item_Identifier'].astype(str))
#test_X['Item_Identifier'] = number.fit_transform(test_X['Item_Identifier'].astype(str))

In [100]:
df['Outlet_Size'].replace(3, 0, inplace=True)

test['Outlet_Size'].replace(3, 0, inplace=True)

In [101]:
median = df_X['Outlet_Size'].mode()
df_X['Outlet_Size'].fillna(median, inplace=True)

median = test_X['Outlet_Size'].mode()
test_X['Outlet_Size'].fillna(median, inplace=True)

In [102]:
test_X

,Unnamed: 0,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,low_fat
0,0,FDW58,20.750,0.007565,13,107.8622,9,1,0,1,1
1,1,FDW14,8.300,0.038428,4,87.3198,2,3,1,1,0
2,2,NCN55,14.600,0.099575,11,241.7538,0,3,2,0,1
3,3,FDQ58,7.315,0.015388,13,155.0340,2,3,1,1,1
4,4,FDY38,NaN,0.118599,4,234.2300,5,1,2,3,0
...,...,...,...,...,...,...,...,...,...,...,...
5676,5676,FDB58,10.500,0.013496,13,141.3154,8,2,0,1,0
5677,5677,FDD47,7.600,0.142991,15,169.1448,3,1,2,2,0
5678,5678,NCO17,10.000,0.073529,8,118.7440,7,3,1,1,1
5679,5679,FDJ26,15.300,0.000000,3,214.6218,2,3,1,1,0


In [103]:
df1 = df_X.groupby('Item_Identifier')['Item_Weight'].mean()
rf1 = pd.merge(df_X,df1,on='Item_Identifier')
rf1.dropna(subset = ["Item_Weight_y"], inplace=True)
df_X = rf1.drop(['Item_Weight_x','Item_Identifier'],axis=1)

df2 = test_X.groupby('Item_Identifier')['Item_Weight'].mean()
rf2 = pd.merge(test_X,df2,on='Item_Identifier')
rf2.dropna(subset = ["Item_Weight_y"], inplace=True)
test_X = rf2.drop(['Item_Weight_x','Item_Identifier'],axis=1)

In [104]:
test_X

,Unnamed: 0,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,low_fat,Item_Weight_y
0,0,0.007565,13,107.8622,9,1,0,1,1,20.750
1,253,0.007596,13,104.4622,2,3,1,1,1,20.750
2,1625,0.007584,13,107.0622,3,1,2,2,1,20.750
3,1892,0.000000,13,105.9622,8,2,0,1,1,20.750
4,2166,0.007568,13,105.8622,7,3,1,1,1,20.750
...,...,...,...,...,...,...,...,...,...,...
5673,5312,0.030089,3,58.3220,8,2,0,1,0,13.000
5674,5556,0.000000,3,60.5220,9,1,0,1,0,13.000
5676,5553,0.014109,5,179.4344,9,1,0,1,0,11.800
5677,5573,0.095944,5,125.8678,9,1,0,1,1,5.985


In [105]:
test_X.to_csv('big7.csv')

In [63]:
#df_X.loc[df['Item_Visibility'] <= 0, 'visible'] = 'False' 
#df_X.loc[df['Item_Visibility'] > 0, 'visible'] = 'True' 
#test_X.loc[df['Item_Visibility'] <= 0, 'visible'] = 'False' 
#test_X.loc[df['Item_Visibility'] > 0, 'visible'] = 'True' 

#df_X['visible'] = number.fit_transform(df_X['visible'].astype(str))
#test_X['visible'] = number.fit_transform(test_X['visible'].astype(str))

In [83]:
test_X  = test_X.sort_values(by=['Unnamed: 0'])

In [94]:
test_X

,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,low_fat,Item_Weight_y,ans
0,0.007565,13,107.8622,9,1,0,1,1,20.750,1831.582510
7,0.038428,4,87.3198,2,3,1,1,0,8.300,1188.985640
13,0.099575,11,241.7538,0,3,2,0,1,14.600,569.225710
17,0.015388,13,155.0340,2,3,1,1,1,7.315,1919.554664
20,0.118599,4,234.2300,5,1,2,3,0,13.600,6953.621858
...,...,...,...,...,...,...,...,...,...,...
4874,0.013496,13,141.3154,8,2,0,1,0,10.500,2377.531852
3610,0.142991,15,169.1448,3,1,2,2,0,7.600,2752.816680
5314,0.073529,8,118.7440,7,3,1,1,1,10.000,1942.424894
4182,0.000000,3,214.6218,2,3,1,1,0,15.300,3913.172920


In [84]:
df_Y = df_X['Item_Outlet_Sales']
df_X = df_X.drop(['Item_Outlet_Sales','Unnamed: 0'],axis=1)
test_X = test_X.drop(['Unnamed: 0'],axis=1)

In [92]:


#############################-----TRAIN-----############################
# splitting X and y into training and testing sets 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.3, 
                                                    random_state=1) 
  
# create linear regression object 
reg = linear_model.LinearRegression() 
ran = RandomForestRegressor()
xgr = xgb.XGBRegressor()
# train the model using the training sets 
reg.fit(X_train, y_train) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [93]:
Pipeline_1 = Pipeline([('pca1',reg)])

Pipeline_2 = Pipeline([('pca2',ran)])

Pipeline_3 = Pipeline([('pca3',xgr)])


pipeline = [Pipeline_1,Pipeline_2,Pipeline_3]

pipe_dict = {0:'Pipeline_1', 1:'e ipeline_2'}

for pipe in pipeline:
    pipe.fit(X_train, y_train)
    ans1 =  pipe.predict(X_test)
    #print(mean_squared_error(y_test, ans1))
    rmse = np.sqrt(mean_squared_error(y_test, ans1))
    print("RMSE: %f" % (rmse))

RMSE: 1208.637651
RMSE: 1121.970308
[14:35:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 1087.173510


In [87]:

#ans1 =  xgr.predict(test_X)
ans1 =  ran.predict(test_X)

In [88]:
test_X['ans'] = ans1

In [89]:
test_X['ans']

0       1831.582510
7       1188.985640
13       569.225710
17      1919.554664
20      6953.621858
           ...     
4874    2377.531852
3610    2752.816680
5314    1942.424894
4182    3913.172920
1759    1572.373254
Name: ans, Length: 5661, dtype: float64

In [90]:
test_X.to_csv('big7_ans.csv')

In [ ]:
frequency = df['department'].value_counts(normalize =True)

In [91]:
test_X

,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,low_fat,Item_Weight_y,ans
0,0.007565,13,107.8622,9,1,0,1,1,20.750,1831.582510
7,0.038428,4,87.3198,2,3,1,1,0,8.300,1188.985640
13,0.099575,11,241.7538,0,3,2,0,1,14.600,569.225710
17,0.015388,13,155.0340,2,3,1,1,1,7.315,1919.554664
20,0.118599,4,234.2300,5,1,2,3,0,13.600,6953.621858
...,...,...,...,...,...,...,...,...,...,...
4874,0.013496,13,141.3154,8,2,0,1,0,10.500,2377.531852
3610,0.142991,15,169.1448,3,1,2,2,0,7.600,2752.816680
5314,0.073529,8,118.7440,7,3,1,1,1,10.000,1942.424894
4182,0.000000,3,214.6218,2,3,1,1,0,15.300,3913.172920


In [95]:
test_X

,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,low_fat,Item_Weight_y,ans
0,0.007565,13,107.8622,9,1,0,1,1,20.750,1831.582510
7,0.038428,4,87.3198,2,3,1,1,0,8.300,1188.985640
13,0.099575,11,241.7538,0,3,2,0,1,14.600,569.225710
17,0.015388,13,155.0340,2,3,1,1,1,7.315,1919.554664
20,0.118599,4,234.2300,5,1,2,3,0,13.600,6953.621858
...,...,...,...,...,...,...,...,...,...,...
4874,0.013496,13,141.3154,8,2,0,1,0,10.500,2377.531852
3610,0.142991,15,169.1448,3,1,2,2,0,7.600,2752.816680
5314,0.073529,8,118.7440,7,3,1,1,1,10.000,1942.424894
4182,0.000000,3,214.6218,2,3,1,1,0,15.300,3913.172920
